In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

AWS_ACCESS_KEY = os.getenv("ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("SECRET_KEY")

WEAVIATE_CLOUD_URL = os.getenv("WEAVIATE_CLOUD_URL")
WEAVIATE_CLOUD_KEY = os.getenv("WEAVIATE_CLOUD_KEY_ADMIN")

## Connect


In [ ]:
import weaviate, json

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLOUD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_CLOUD_KEY),

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
    }
)

# Connect to the local instance deployed with Docker Compose
# client = weaviate.connect_to_local(
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     }
# )

client.is_ready()

## Create collection with a gerative model

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    # Configure Vectorizer - with AWS vectorizer and Titan model("amazon.titan-embed-text-v1")
    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        # region="eu-central-1",
        region="us-east-1",
    ),
    
    # Configure Generative model with AWS "cohere.command-r-plus-v1:0" model 
    generative_config=Configure.Generative.aws(
        region="us-east-1",
        model="cohere.command-r-plus-v1:0"
    ),

    properties=[  # Define properties (Optional)
        Property(name="chunk", data_type=DataType.TEXT),
        Property(name="arxiv_id", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="title", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="url", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="chunk_no", data_type=DataType.INT),
    ],
)

## Close the client when done

In [ ]:
client.close()